# Ejercicios Pair Programming 
## Lección 10 - Métricas Regresión Lineal

En el pair programming anterior creastéis vuestro primer modelo de machine learning usando la regresion Lineal. Es el momento, que con vuestros datos evaluéis si es bueno haciendo predicciones. Los objetivo de este pairprogramming son:

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [9]:
df = pd.read_csv("supermercado_normalizado.csv", index_col=0)

In [10]:
df.head()

,id,year_birth,education,marital_status,income,dt_customer,numdealspurchases,numstorepurchases,numwebvisitsmonth,numwebpurchases_sklearn
0,1826,1970,Graduation,Divorced,84835.0,6/16/2014,1,6,1,0.363636
1,1,1961,Graduation,Single,57091.0,6/15/2014,1,7,5,0.636364
2,10476,1958,Graduation,Married,67267.0,5/13/2014,1,5,2,0.272727
3,1386,1967,Graduation,Together,32474.0,11/5/2014,1,2,7,0.090909
4,5371,1989,Graduation,Single,21474.0,8/4/2014,2,2,7,0.272727


Calculéis las métricas para vuestro modelo

Discutid los resultados de las métricas y extraed conclusiones

Guardad los resultados de las métricas en un csv para usarlo más adelante.

Happy coding💪🏽